In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

import matplotlib.pyplot as plt
from scipy.stats import mode
import random
import pickle
from multiprocessing import Pool
import gc
from tqdm import tqdm

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

from keras import models, layers, losses, optimizers, Model
from keras.utils import plot_model

2023-11-08 14:03:31.890035: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 14:03:31.890114: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 14:03:31.890161: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
gc.collect()

0

In [3]:
def get_feature_vectors():
    with open("feature_vecs.txt", "r") as f:
        f = f.read().split("\n\n\n")
        noun_dict = {}
        for noun_vec in f:
            split = noun_vec.split("\n\n")
            noun = split[0].split()[2][:-1]
            
            vec = split[1]
            vec = [item.strip().replace("(", "").replace(")", "") for item in vec.split(",\n")]
            vec = [(item.split()[:-1], item.split()[-1]) for item in vec]

            vec = sorted(vec, key=lambda x: x[0])
            vec = {" ".join(item[0]): float(item[1]) for item in vec}

            noun_dict[noun] = vec

    return {k: [noun_dict[k][k1] for k1 in noun_dict[k]] for k in noun_dict}


In [4]:
NUM_CLASSES = 60

pickles = [pickle.load(open(f"pickles/{i}.pkl", "rb")) for i in range(1)]
pickles = [item for sublist in pickles for item in sublist]
pickles = sorted(pickles, key=lambda x: x[1])

feature_vectors = get_feature_vectors()

nouns = list(set([item[1] for item in pickles]))

pickles = [[item for item in pickles if item[1] == noun] for noun in nouns]
pickles = [(np.add.reduce([item[0] for item in sublist]) / len(sublist), sublist[0][1]) for sublist in pickles]

In [5]:
def standardize(x):
    x = (x - x.min()) / (x.max() - x.min())

In [6]:
def plot_scan(scan):
    scan[scan == mode(scan.flat).mode] = -1
    w = 5
    fig, ax = plt.subplots(w, w, constrained_layout=True)
    fig.dpi = 100
    bg_color = (225 / 255, 216 / 255, 226 / 255)
    fig.set_facecolor(bg_color)

    for j in range(w * w):
        ax[(j - j % w) // w, j % w].imshow(scan[j % scan.shape[0]], vmin=-1, vmax=1, cmap="twilight")
        ax[(j - j % w) // w, j % w].set_xticks([])
        ax[(j - j % w) // w, j % w].set_yticks([])
        plt.setp(ax[(j - j % w) // w, j % w].spines.values(), color=bg_color)

In [7]:
def double_plot(scan1, scan2):
    scan1 = standardize(scan1)
    scan2 = standardize(scan2)

    scan1 = (scan1 - 0.5) * 2
    scan2 = (scan2 - 0.5) * 2

    scan1[scan1 == mode(scan1.flat).mode] = -1
    scan2[scan2 == mode(scan2.flat).mode] = -1

    w = 5
    fig, ax = plt.subplots(w, 2 * w, constrained_layout=True)
    fig.dpi = 100
    bg_color = (225 / 255, 216 / 255, 226 / 255)
    fig.set_facecolor(bg_color)

    for j in range(w * w):
        ax[(j - j % w) // w, j % w].imshow(scan1[j % scan1.shape[0]], vmin=-1, vmax=1, cmap="twilight")
        ax[(j - j % w) // w, j % w].set_xticks([])
        ax[(j - j % w) // w, j % w].set_yticks([])
        plt.setp(ax[(j - j % w) // w, j % w].spines.values(), color=bg_color)

        ax[(j - j % w) // w, j % w + w].imshow(scan2[j % scan2.shape[0]], vmin=-1, vmax=1, cmap="twilight")
        ax[(j - j % w) // w, j % w + w].set_xticks([])
        ax[(j - j % w) // w, j % w + w].set_yticks([])
        plt.setp(ax[(j - j % w) // w, j % w + w].spines.values(), color=bg_color)

In [8]:
class BasisSum(Model):
    def __init__(self):
        super().__init__()
        self.basis = tf.Variable(tf.zeros((25, 23, 61, 51, 1)))

    @tf.function(reduce_retracing=True)
    def call(self, x):
        x = tf.einsum("ijklm,bim->bjklm", self.basis, x)

        return x

In [9]:
def cos_sim(a, b):
    return dot(a, b) / (norm(a) * norm(b))

In [10]:
model = BasisSum()

loss = losses.MeanSquaredError()
opt = optimizers.Adam(0.01)

total = 500
pbar = tqdm(range(total))
correct_count = 0

for i in pbar:
    random.shuffle(pickles)

    x = np.array([item[1] for item in pickles])
    x = [feature_vectors[item] for item in x]
    x = np.array([item / np.linalg.norm(item) for item in x])
    y = np.array([item[0] for item in pickles])
    y_mean = np.add.reduce(y) / len(y)
    y -= y_mean

    x, y = np.expand_dims(x, -1), np.expand_dims(y, -1)
    x, y = tf.cast(x, tf.dtypes.float32), tf.cast(y, tf.dtypes.float32)
    
    train_x, test_x = x[:-2], x[-2:]
    train_y, test_y = y[:-2], y[-2:]

    for j in range(100):
        with tf.GradientTape() as tape:
            pred_y = model(train_x)
            batchloss = loss(train_y, pred_y)

            grad = tape.gradient(batchloss, model.trainable_variables)
            opt.apply_gradients(zip(grad, model.trainable_variables))


    pred = model(test_x)
    t1, t2 = test_y.numpy()
    t1, t2 = t1.flat, t2.flat
    p1, p2 = pred.numpy()
    p1, p2 = p1.flat, p2.flat
    
    correct = cos_sim(t1, p1) + cos_sim(t2, p2) > cos_sim(t1, p2) + cos_sim(t2, p1)
    correct_count += int(correct)

    pbar.set_description(f"accuracy: {correct_count / (i + 1):.3f}")

0.954: 100%|██████████| 500/500 [12:44<00:00,  1.53s/it]             
